In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

book = pd.read_csv("delicious_csv/bookmarks.csv")
book_tags = pd.read_csv("delicious_csv/bookmark_tags.csv")
tags = pd.read_csv("delicious_csv/tags.csv")
user_contacts = pd.read_csv("delicious_csv/user_contacts.csv")
user_contacts_t = pd.read_csv("delicious_csv/user_contacts_timestamps.csv")
user_taggedb = pd.read_csv("delicious_csv/user_taggedbookmarks.csv")
user_taggedbt = pd.read_csv("delicious_csv/user_taggedbookmarks_timestamps.csv")

In [ ]:
idx_to_user = list(set(user_taggedbt["userID"]))

useridx_to_book = []
for idx, u in enumerate(idx_to_user):
    useridx_to_book.append(list(user_taggedbt[user_taggedbt["userID"] == u]["bookmarkID"]))
    
idx_to_book = list(book["id"])

# there can be some items receving more tags ... some less
# but here, let's just say --> 1, 0

num_items = len(idx_to_book)

In [ ]:
pca = np.load("tfidf_pca_25_inc.npy")

In [ ]:
def get_contexts(user_idx, num_contexts):
    
    items = useridx_to_book[user_idx]
    items_idx = np.array([idx_to_book.index(item) for item in items])
    
    good_item_context = pca[np.random.choice(items_idx)]
    bad_items_idx = np.random.choice(np.setdiff1d(range(num_items), items_idx), num_contexts - 1) 
    bad_items_contexts = pca[bad_items_idx]
    
    contexts = np.vstack((good_item_context, bad_items_contexts))
    rewards = np.array([1] + 24 * [-1/24])
    
    return contexts, rewards

In [ ]:
# hyperparams
N_u = len(idx_to_user)
K = 1

d = 25 # feature length
num_contexts = 25
alpha = 0.5
Lambda = 0.01

T = 20 # no. of iterations

In [ ]:
# initialize

b = np.zeros((N_u, d))
w = np.zeros((N_u, d))

M = np.empty((N_u, d, d))
for u in range(N_u):
    M[u] = np.eye(d, d)

r = np.random.choice(K, N_u) 

C = [np.where(r == k)[0] for k in range(K)]

M_cap = np.empty((K, d, d))
b_cap = np.zeros((K, d))
w_cap = np.empty((K, d))

# this loop can be removed... vectorize
for k in range(K):
    b_cap[k] = np.sum(b[C[k]], axis = 0)
    M_cap[k] = np.eye(d, d) + np.sum(M[C[k]] - np.eye(d, d), axis = 0)
    w_cap[k] = np.linalg.inv(M_cap[k]) @ b_cap[k]
  # check context dep clustering paper.. a better way to update


In [ ]:
for it in range(1, T + 1):
    if it % 2 == 0: print(it)
    u = np.random.choice(N_u)
    k = r[u]
    contexts, rewards = get_contexts(u, num_contexts)
    
    a = np.argmax(contexts @ w_cap[k] + alpha * np.sqrt(np.log(it + 1) * np.diag(contexts @ np.linalg.inv(M_cap[k]) @ contexts.T)))
    r_obs = rewards[a]
    
    M[u] += contexts[a] @ contexts[a].T
    b[u] += r_obs * contexts[a]
    w[u] = np.linalg.inv(M[u]) @ b[u] # check better ways
    
    # reassign clusters
    k_new = np.argmin(np.linalg.norm(w[u] - w_cap, axis = 1))
    if k_new != k:
        r[u] = k_new
        C[k_new] = np.append(C[k_new], u)
        C[k] = np.setdiff1d(C[k], u)
        
        # recompute vals for k and k_new
        # for k
        b_cap[k] -= b[u]
        M_cap[k] -= M[u] - np.eye(d, d)

        # for k_new
        b_cap[k_new] += b[u]
        M_cap[k_new] += M[u] - np.eye(d, d)

        for idx in [k, k_new]:
            w_cap[idx] = np.linalg.inv(M_cap[idx]) @ b_cap[idx]

2
4
6
8
10
12
14
16
18
20


In [ ]:
# dp-means
for it in range(1, T + 1):
    if it % 2 == 0: print(it)
    u = np.random.choice(N_u)
    k = r[u]
    contexts, rewards = get_contexts(u, num_contexts)
    
    a = np.argmax(contexts @ w_cap[k] + alpha * np.sqrt(np.log(it + 1) * np.diag(contexts @ np.linalg.inv(M_cap[k]) @ contexts.T)))
    r_obs = rewards[a]
    
    M[u] += contexts[a] @ contexts[a].T
    b[u] += r_obs * contexts[a]
    w[u] = np.linalg.inv(M[u]) @ b[u] # check better ways
    
    # reassign clusters
    dists = np.linalg.norm(w[u] - w_cap, axis = 1)
    min_val = np.min(dists)
    print(min_val)
    if min_val <= Lambda:
        k_new = np.argmin(dists)
        if k_new != k:
            r[u] = k_new
            C[k_new] = np.append(C[k_new], u)
            C[k] = np.setdiff1d(C[k], u)

            # recompute vals for k and k_new
            # for k
            b_cap[k] -= b[u]
            M_cap[k] -= M[u] - np.eye(d, d)
            
            # for k_new
            b_cap[k_new] += b[u]
            M_cap[k_new] += M[u] - np.eye(d, d)
            
            for idx in [k, k_new]:
                w_cap[idx] = np.linalg.inv(M_cap[idx]) @ b_cap[idx]
                
    else:
        K = K + 1
        r[u] = K
        C.append(np.array(u)) # add a new set
        C[k] = np.setdiff1d(C[k], u) # remove u from older set
        w_cap = np.vstack((w_cap, w[u]))
        M_cap = np.vstack((M_cap, M[u][None, :]))
        b_cap = np.vstack((b_cap, b[u]))
        
        # recompute only for k
        b_cap[k] -= b[u]
        M_cap[k] -= M[u] - np.eye(d, d)
        w_cap[k] = np.linalg.inv(M_cap[k]) @ b_cap[k]
        # can refactor the code ... 

0.031147002914085037
2
0.047946840829768196
0.04629488490735968
4
0.0500098979925474
0.04678132837488875
6
0.010862433580182525
0.029196162390157192
8
0.02036548351599269
0.011653465905799012
10
0.034663394007870406
0.00789339145971224
12
0.013016242446942647
0.02958742505205972
14
0.011189475221607237
0.01391586942099356
16
0.012705912645315563
0.02149769983768524
18
0.0
0.03501838496674674
20
0.04506383417148352


In [ ]:
C

[array([   0,    1,    2, ..., 1864, 1865, 1866]),
 array(844),
 array(200),
 array(14),
 array(1007),
 array(1317),
 array(1303),
 array(1093),
 array(1762),
 array([1821, 1381]),
 array(1848),
 array(1337),
 array(1132),
 array([ 689, 1172]),
 array(1428),
 array(1020),
 array(1292),
 array(1248),
 array(1747)]